In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("LinearRegression") \
.master("local[4]") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

# Reading Data Set

In [5]:
df = spark.read.format("csv") \
.option("header",True) \
.option("sep",",") \
.option("inferSchema",True) \
.load("Q:\\SparkDatasets\\Advertising.csv")

In [6]:
df.toPandas().head()

,_c0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


**// Sales (Satışlar) bizim hedef değişkenimiz. "Tv" , "Radio" , "Newspaper" bu üçü ise reklam bütçesi.**

**// Reklam bütçesi ile bu satış "Sales" değişkenini tahmin etmeye çalışacağız."**

**// Fakat biz "Tv" , "Radio" , "Newspaper" değişkeni girdi olarak sokmayacağız çünkü o zaman "Çoklu Linear Regresyon" olur ki o bizim ileriki konumuz.**

**// O yuzden bu üçünü toplayacağım ve bu üçünün toplamından yeni bir sütun oluşturacağım ve adınada Advertisement diyeceğim ki böylece tek bir girdi ve tekbir çıktı olacak ki dolayısıyla basit linear regresyon uygulamış olacağız.**

In [7]:
df2 = df.withColumn("Advertisement", df.TV + df.Radio + df.Newspaper) \
.withColumnRenamed("Sales","label") \
.drop("TV","Radio","Newspaper")

df2.toPandas().head()

,_c0,label,Advertisement
0,1,22.1,337.1
1,2,10.4,128.9
2,3,9.3,132.4
3,4,18.5,251.3
4,5,12.9,250.0


# Data Preparing And Discovering

In [8]:
df2.describe("label","Advertisement").toPandas().head()

,summary,label,Advertisement
0,count,200,200
1,mean,14.022500000000003,200.86049999999992
2,stddev,5.217456565710477,92.9851805869837
3,min,1.6,11.7
4,max,27.0,433.6


**// count : toplam kayıt sayısı , mean : ortalama , stddev : standart sapma , min : min , max : max**

**// Kategorik değişkenimiz olmadıgından onehotencoder ve stringindexer yapmayacağız bunlara yönelik bir hazılık yapmayacağız.**

# VectorAssembler

In [9]:
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler() \
.setInputCols(["Advertisement"]) \
.setOutputCol("features")

**//setInputCols "s" takısına dikkat çoğul oldugundan girdiyi liste halinde vermeniz gerekmekte.//**

**//Inputumuz (Girdi değişkenimiz "Advertisement" Output değişkenimize de features diyelim. )**

# Regresyon Model

In [10]:
from pyspark.ml.regression import LinearRegression
linear_reg_obj = LinearRegression() \
.setLabelCol("label")\
.setFeaturesCol("features")

# pipeline

In [11]:
from pyspark.ml import Pipeline
pipeline_obj = Pipeline() \
.setStages([vector_assembler, linear_reg_obj])

**Pipeline Stages 'ler alır ve ".setStages([])" çogul oldugundan liste içerisinde alır ve bunları baglar.**

# Sperate Dataset

In [12]:
train_df, test_df = df2.randomSplit([0.8, 0.2], seed=142)

# Training Model

In [13]:
pipeline_model = pipeline_obj.fit(train_df)

# Model Transform With Test Data (Modeli Test Etme)

In [14]:
result_df = pipeline_model.transform(test_df)

In [15]:
result_df.toPandas().head(200)

,_c0,label,Advertisement,features,prediction
0,3,9.3,132.4,[132.39999999999998],10.791216
1,6,7.2,132.6,[132.6],10.800663
2,9,4.8,11.7,[11.7],5.089785
3,10,10.6,223.6,[223.6],15.099174
4,17,12.5,218.4,[218.4],14.853545
5,27,15.0,184.8,[184.8],13.266402
6,31,21.4,364.4,[364.4],21.750056
7,34,17.4,285.9,[285.90000000000003],18.042000
8,35,9.5,104.5,[104.50000000000001],9.473321
9,41,16.6,256.4,[256.40000000000003],16.648527


**// Suan bu yaptıgımız tahminin önümüzdeki adımlarda ne kadar iyi ne kadar kötü olduguna bakacağız ama muhtemelen ortalamadan daha iyi bir tahminde bulunduk diye düşünüyorum.**

In [16]:
#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import seaborn as sns
#import matplotlib.pyplot as plt
#from collections import Counter
#%matplotlib inline

In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# word cloud library
#from wordcloud import WordCloud

# matplotlib
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [18]:
pandas_df = result_df.select("*").toPandas()


In [19]:
# Creating trace1
trace1 = go.Scatter(
                    x = pandas_df.Advertisement,
                    y = pandas_df.label,
                    mode = "lines+markers",
                    name = "sales",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= "sales information")

In [20]:
 # Creating trace2
trace2 = go.Scatter(
                    x = pandas_df.Advertisement,
                    y = pandas_df.prediction,
                    mode = "lines+markers",
                    name = "prediction",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "prediction information")

In [21]:
data = [trace1, trace2]
layout = dict(title = 'Efficieny of Advertisement on Sales',
              xaxis= dict(title= 'Advertisement',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)

In [22]:
# prepare data frames

# import graph objects as "go"
import plotly.graph_objs as go
# creating trace1
trace1 =go.Scatter(
                    x = pandas_df.Advertisement,
                    y = pandas_df.label,
                    mode = "markers",
                    name = "Sales",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text= "Sales Information")
# creating trace2
trace2 =go.Scatter(
                    x = pandas_df.Advertisement,
                    y = pandas_df.prediction,
                    mode = "markers",
                    name = "Prediction",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text= "Prediction Information")

data = [trace1, trace2]
layout = dict(title = 'Efficieny of Advertisement on Sales',
              xaxis= dict(title= 'Advertisement',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Sales Info',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)

# LinearRegresyon Modelini Pipeline Stages arasından alma

In [23]:
lrModel = pipeline_model.stages[1]

In [24]:
lrModel.coefficients

DenseVector([0.0472])

<span style="color:deeppink">coefficients (katsayımızı) buluyoruz denklemimizdeki </span>

In [25]:
lrModel.summary.r2

0.7204575410444246

In [26]:
lrModel.intercept 

4.537119328969264

<span style="color:deeppink">intercept (sabit) buluyoruz denklemimizdeki </span>

In [27]:
lrModel.summary.r2

0.7204575410444246

<span style="color:deeppink">spark kütüphanesinden r2 ye bakabilirsiniz. </span>

<span style="color:green">r2 nin anlamı şu değişkenliği "1" kabul ettiğimizde bizim modelimiz bu değişkenliğin %72 sini açıklayabiliyormuş. </span>

<span style="color:blue">Bu model anlamlı mı değil mi buna karar vermek için p değerlerine bakacağız. </span>

<span style="color:#17A589">0.0 Bizim Advertisement bütçesinin katsayısı ve anlamlı , 3.1086244689504383e-15 sabitimizde anlamlı eğer anlamlılık derecesini 0.015 den asagıya tutarsak anlamlı bir model olur.</span>

In [28]:
lrModel.summary.pValues

[0.0, 3.1086244689504383e-15]

<span style="color:#E59866">Her bir niteliğin hedef değişkene olan 2 li t testinin istatistikleri  asagıda bunların anlamlılık derecesi ise "pValues" ile gösteriliyor.</span>

In [29]:
lrModel.summary.tValues

[19.986932892719693, 8.769612805482973]

<span style="color:#8E44AD"> rootMeanSquaredError yani hataların karalerinin ortalaması ne kadarmış ona bakalım.</span>

In [30]:
lrModel.summary.rootMeanSquaredError

2.678303634403656

<span style="color:#8E44AD"> Zaten yukarıdan label'a ait stdev e bakarsanız 5.217456565710477. bizim hataların ortalaması 2.678303634403656 standart sapmaya göre düşük bir hata sayılabilir.</span>

In [31]:
# Dogrumuz,
# y=4.537119328969264 + 0.0472 * Advertisement

# Prediction

In [32]:
# 100 bin liralık bir reklam bütçesi ile ne kadar satış gerçekleşeceğini tahmin edelim.
df_predict_rdd = spark.sparkContext.parallelize([100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0,900.0,1000.0])
df_predict = df_predict_rdd.map(lambda x: (x,)).toDF(["Advertisement"])

In [33]:
df_predict.show()

+-------------+
|Advertisement|
+-------------+
|        100.0|
|        200.0|
|        300.0|
|        400.0|
|        500.0|
|        600.0|
|        700.0|
|        800.0|
|        900.0|
|       1000.0|
+-------------+



In [34]:
df_pred_vec = vector_assembler.transform(df_predict)

In [35]:
result=lrModel.transform(df_pred_vec).toPandas()

In [36]:
result.head(10)

,Advertisement,features,prediction
0,100.0,[100.0],9.260757
1,200.0,[200.0],13.984395
2,300.0,[300.0],18.708033
3,400.0,[400.0],23.431671
4,500.0,[500.0],28.155310
5,600.0,[600.0],32.878948
6,700.0,[700.0],37.602586
7,800.0,[800.0],42.326224
8,900.0,[900.0],47.049862
9,1000.0,[1000.0],51.773500


In [37]:
# Creating trace1
trace_predict = go.Scatter(
                    x = result.Advertisement,
                    y = result.prediction,
                    mode = "lines+markers",
                    name = "prediction",
                    marker = dict(color = 'rgba(241, 196, 15, 0.8)'),
                    text= "Advertisement Prediction")

In [38]:
data = [trace_predict]
layout = dict(title = 'New Sales Prediction According To Budget',
              xaxis= dict(title= 'Advertisement',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)